## Setup

In [38]:
from functools import reduce

import mcl
mcl.mcl_init(mcl.CurveType.MCL_BLS12_381)

Q = mcl.G1.BLS12_381_G1_generator()

## User

In [39]:
password = 'cl0udc0mp'
fid = 'my_file.txt'

In [40]:
# Create secret key based on password
sk = mcl.Fr()
sk = sk.set_hash_of(password)
print('Sk:', sk)

Sk: 24357992188968198044373167526530025380864430797887514595416667565948689152304


In [41]:
# Encode file into chunks

m_a_user = []
with open(fid, "rb") as file:
    while True:
        data = file.read(31)
        print(data)
        if not data:
            break
        f = mcl.Fr()
        f.setStr(str(int(data.hex(), base=16)))
        m_a_user.append(f)

b'Lorem ipsum dolor sit amet cons'
b'ectetur adipisicing elit. Harum'
b' eum numquam architecto eligend'
b'i quis alias ex, ullam sapiente'
b' voluptatum deleniti animi? Ips'
b'am eveniet alias harum, maiores'
b' vero aliquam quia, necessitati'
b'bus sequi excepturi autem odit.'
b' Soluta id possimus doloribus t'
b'empore assumenda. Reiciendis is'
b'te fuga dolorem voluptatibus od'
b'it animi, accusamus alias sed q'
b'uos deleniti soluta delectus, n'
b'ihil eos aliquam facilis quod, '
b'eveniet minima ipsam sequi atqu'
b'e explicabo. Praesentium eos qu'
b'am officia veniam laborum numqu'
b'am quaerat. Perferendis fugit d'
b'istinctio numquam eveniet quod '
b'voluptates, totam consequatur v'
b'oluptatem ipsum natus ut mollit'
b'ia praesentium deleniti invento'
b're dolor rerum, facilis officia'
b' hic blanditiis porro, veniam e'
b'os amet asperiores enim. Enim c'
b'onsequuntur numquam cumque dolo'
b'ribus quos laboriosam voluptate'
b'm similique fugiat, commodi par'
b'iatur eaque sequi,

In [42]:
print('m_i encoded chunks:', *m_a_user, sep='\n')

m_i encoded chunks:
135049554883004558383340439742929429255072943744440858662311072526922378867
179137964469212291398418277116391414980772671563800423105930264587936101741
57239348260075530734666324302070080995136383133407562338710055187856584292
185742856523718661651775617568384026070692213914768531576669899575863964773
57356516117948854894767387608989504901290688788449954977284834861618131059
172137329038843637016105995424813767226241267411913634180951755338156238195
57356247147759002523921200487803187254627411165702287562049947908232541289
173961620482187484936551088381765961816589305672770623715554745305694565422
57114954995817492054595809049321805949380802598249462758622147028194631796
179206875145167468705119314083914247211291423999163068888070792696419084659
205652209403762660780829680762537122034077256079041105654913051064897924964
186320417356964491279997711933513269442095484569623101017775340497066532977
207490304228725369948633313542071047987266669713741304105426820084508926

In [43]:
# Create polynomial L
L_coef = [mcl.Fr().set_hash_of(f'{sk}{fid}{i}') for i in range(len(m_a_user))]
polynomial_core = lambda c_a: lambda x: reduce(lambda acc, val: acc * x + val, c_a)
assert(polynomial_core([1, 2, 3])(5) == 1 * 5 * 5 + 2 * 5 + 3)
L = polynomial_core(L_coef)

# print('L coefficients:', *L_coef, sep='\n')

In [44]:
# Create tags for chunks
t_a_user = [L(m) for m in m_a_user]
print('Pairs (m_i, t_i):', *list(zip(m_a_user, t_a_user)), sep='\n')

Pairs (m_i, t_i):
(135049554883004558383340439742929429255072943744440858662311072526922378867, 8373934017232533746134815804918727087931502677049918695789669211843030345519)
(179137964469212291398418277116391414980772671563800423105930264587936101741, 18134819249510293026016369816654777727279938123533525766476473625951812712057)
(57239348260075530734666324302070080995136383133407562338710055187856584292, 38892825145079552238654718195546034739462654013943890002259851594739019123628)
(185742856523718661651775617568384026070692213914768531576669899575863964773, 49256694046829431495998108421052070524809214162000718109097756786927134248111)
(57356516117948854894767387608989504901290688788449954977284834861618131059, 33914216114327718765852563552232997852839455866629232708536000626059240425311)
(172137329038843637016105995424813767226241267411913634180951755338156238195, 17866767948304098565775927183529252156577153059134046606992086323526018815416)
(573562471477590025239212004878031872546274

## Cloud

In [45]:
m_a_cloud = list(m_a_user)
t_a_cloud = list(t_a_user)

## Challenge

In [46]:
# Generate random generator
r = mcl.Fr()
r.set_by_CSPRNG()
Qr = mcl.G1()
Qr = Q * r
print('Qr:', Qr)

Qr: 1 921806092078183517175776109632020792886505468018743606693788291565621083787385625376192486924642810282987745508523 3062283049407381842503042423031670759078712704827290500328527309911087182667126511670927723890197411359091119248129


In [47]:
# Generate challenge
x_c = mcl.Fr()
while True:
    x_c.set_by_CSPRNG()
    if x_c not in m_a_user:
        break
print('x_c:', x_c)

x_c: 51713326881664240201353155116077383349593000234930904708371247776591558971389


In [48]:
# Calculate Qr ^ L(0)
QrL0 = mcl.G1()
QrL0 = Qr * L(mcl.Fr())
print('QrL0:', QrL0)

QrL0: 1 2842034360201236730225404378523469098578548096971041111963328896012132884888604720701709347757428749702599668023901 703956046730330064955345344881712578098652644791271586886676123516609541988799417828073630806132558912422880607255


In [49]:
# Calculate expected response R_u = Qr ^ L(x_c)
R_u = Qr * L(x_c)
print('R_u:', R_u)

R_u: 1 1736629941363967896430600138953657818305677421289054593604811447235136798211935683924721434819763522539104607194910 1561155987812827615738219194106587413887977008447303796726769146235149548079218253552901678611380491713621699472487


## Response

In [50]:
Qr_cloud = Qr
x_c_cloud = x_c
QrL0_cloud = QrL0

In [51]:
# Prepare interpolation set
m_a_cloud_tmp = list(m_a_cloud)
t_a_cloud_tmp = [Qr_cloud * t for t in t_a_cloud]
m_a_cloud_tmp.append(mcl.Fr())
t_a_cloud_tmp.append(QrL0)
print('Pairs (m_i, t*_i):', *list(zip(m_a_cloud_tmp, t_a_cloud_tmp)), sep='\n')

Pairs (m_i, t*_i):
(135049554883004558383340439742929429255072943744440858662311072526922378867, 1 1134726545097223939630755488040673447503845885225369137370929345680349493949539973470716491446057695040964344398692 1554469573396151765716257037298477794777700945304195864199521511163236757416296051619778817387165357670105328938999)
(179137964469212291398418277116391414980772671563800423105930264587936101741, 1 1502094551097604346840095127522291268011946805236616294145449413454257110007899626781979227887264808720599865437002 3726359611116369776131488449415398612084431510428057228869258882618448784403109561431146238797819352935167262880117)
(57239348260075530734666324302070080995136383133407562338710055187856584292, 1 917315809604858288394269781699262898496073128247980519252458211695972135077992560092180595072256185321690996313171 3049366273961401418517819525132495640683416436904329347407737739880435047090800986968744505405831896408853272366304)
(1857428565237186616517756175683840260706922

In [52]:
# Interpolate L with use of LI_EXP, calculate given response R_c = LI_EXP(x_c)

# Get m_a_cloud_tmp array without an element on given index i
filter_i = lambda a: lambda i: [val for ind, val in enumerate(a) if ind != i]
assert(filter_i([1, 2, 3, 4])(2) == [1, 2, 4])

LI_core = lambda x_a, i, s_0: lambda x: reduce(lambda acc, val: ((x - val) / (x_a[i] - val)) * acc, filter_i(x_a)(i), s_0)
assert(LI_core([1, 2, 3], 1, 1)(5) == ((5 - 1) / (2 - 1)) * (5 - 3) / (2 - 3))

LI_EXP = lambda x_a, y_a, s_0, s_1: lambda x: reduce(lambda acc, val: (val[1] * LI_core(x_a, val[0], s_0)(x)) + acc, enumerate(y_a), s_1)
FR_ONE = mcl.Fr()
FR_ONE.setInt(1)
R_c = LI_EXP(m_a_cloud_tmp, t_a_cloud_tmp, FR_ONE, mcl.G1())(x_c)

print('R_c:', R_c)

R_c: 1 1736629941363967896430600138953657818305677421289054593604811447235136798211935683924721434819763522539104607194910 1561155987812827615738219194106587413887977008447303796726769146235149548079218253552901678611380491713621699472487


## Verify

In [53]:
print('R_u:', R_u)
print('R_c:', R_c)
if R_u == R_c:
    print('Verified')
else:
    print('Rejected')

R_u: 1 1736629941363967896430600138953657818305677421289054593604811447235136798211935683924721434819763522539104607194910 1561155987812827615738219194106587413887977008447303796726769146235149548079218253552901678611380491713621699472487
R_c: 1 1736629941363967896430600138953657818305677421289054593604811447235136798211935683924721434819763522539104607194910 1561155987812827615738219194106587413887977008447303796726769146235149548079218253552901678611380491713621699472487
Verified


## Download

In [54]:
m_a_download = list(m_a_cloud)

In [55]:
# Decode downloaded blocks
with open("dec_" + fid, "wb") as file:
    for m in m_a_download:
        byte_data = int(m.getStr()).to_bytes(31, byteorder='big')
        print(byte_data.replace(b'\x00', b''))
        file.write(byte_data.replace(b'\x00', b''))

b'Lorem ipsum dolor sit amet cons'
b'ectetur adipisicing elit. Harum'
b' eum numquam architecto eligend'
b'i quis alias ex, ullam sapiente'
b' voluptatum deleniti animi? Ips'
b'am eveniet alias harum, maiores'
b' vero aliquam quia, necessitati'
b'bus sequi excepturi autem odit.'
b' Soluta id possimus doloribus t'
b'empore assumenda. Reiciendis is'
b'te fuga dolorem voluptatibus od'
b'it animi, accusamus alias sed q'
b'uos deleniti soluta delectus, n'
b'ihil eos aliquam facilis quod, '
b'eveniet minima ipsam sequi atqu'
b'e explicabo. Praesentium eos qu'
b'am officia veniam laborum numqu'
b'am quaerat. Perferendis fugit d'
b'istinctio numquam eveniet quod '
b'voluptates, totam consequatur v'
b'oluptatem ipsum natus ut mollit'
b'ia praesentium deleniti invento'
b're dolor rerum, facilis officia'
b' hic blanditiis porro, veniam e'
b'os amet asperiores enim. Enim c'
b'onsequuntur numquam cumque dolo'
b'ribus quos laboriosam voluptate'
b'm similique fugiat, commodi par'
b'iatur eaque sequi,